# ML Pipeline Preparation
Here I create a machine learning pipeline.
### 1. Import libraries and load data from database.
The steps I will folow will be: 
- Import the necessary Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import re
import sqlalchemy 

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')
nltk.download('wordnet')

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package words to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
print('The nltk version is {}.'.format(nltk.__version__))
# print('The scikit-learn version is {}.'.format(sklearn.__version__))
print(f'The sklearn version is {sklearn.__version__}')
# print(f'The version of pickle is {pickle.__version__}')

The nltk version is 3.4.4.
The sklearn version is 0.21.2


In [5]:
#I load the data from the database
#sqlite:///
engine = sqlalchemy.create_engine('sqlite:///DisasterResponse.db')

#read it into a Pandas DataFrame
df = pd.read_sql_table('disaster', engine) 

In [6]:
#inspect
df.tail()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
26172,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26173,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26174,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26175,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
26176,30265,A radical shift in thinking came about as a re...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df.shape

(26177, 40)

So there are 26,177 observations and 40 variables, just like in the last notebook. Which is good. Now we get the value counts of the qualitative variables:

In [8]:
for i in ['genre', 'related','request']:
    print(f'the value counts for "{i}" are:\n{df[i].value_counts()}\n')

the value counts for "genre" are:
news      13036
direct    10747
social     2394
Name: genre, dtype: int64

the value counts for "related" are:
1    19873
0     6117
2      187
Name: related, dtype: int64

the value counts for "request" are:
0    21714
1     4463
Name: request, dtype: int64



#### Split the data into independent and dependent variables

In [9]:
X = df.loc[:,'message']
y = df.iloc[:,5:]

In [10]:
X.shape

(26177,)

In [11]:
y.shape

(26177, 35)

In [12]:
type(X)

pandas.core.series.Series

In [13]:
list(y.columns.values)

['request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

### 2. Write a tokenization function to process your text data

In [14]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [15]:
def tokenize(text):
    #normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #tokenize text
    tokens = word_tokenize(text)
    
    #lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [16]:
X_tokenized = X.apply(lambda x: tokenize(x))

In [17]:
X_tokenized.head()

0    [weather, update, cold, front, cuba, could, pa...
1                                          [hurricane]
2                             [looking, someone, name]
3    [un, report, leogane, 80, 90, destroyed, hospi...
4    [say, west, side, haiti, rest, country, today,...
Name: message, dtype: object

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list[0]:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
            
            
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [112]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [41]:
pipeline1 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [46]:
pipeline.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: T

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

In [23]:
y_pred = pipeline.predict(X_test)

In [24]:
accuracy_score(y_test, y_pred)

0.38747135217723455

### 6. Improve your model
Use grid search to find better parameters. 

In [25]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__

In [26]:
parameters = {
        
        'clf__estimator__max_depth': [15, 25],
        'clf__estimator__n_estimators': [50,75] 
    }

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
cv.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [28]:
y_pred = cv.predict(X_test)

In [29]:
cv.best_params_

{'clf__estimator__max_depth': 15, 'clf__estimator__n_estimators': 75}

In [30]:
for i, label in enumerate(y_test.columns):
    print(label)
    print(confusion_matrix(y_pred[:,i] ,y_test.values[:,i]))

request
[[5455 1089]
 [   0    1]]
offer
[[6516   29]
 [   0    0]]
aid_related
[[3887 2151]
 [  51  456]]
medical_help
[[6063  482]
 [   0    0]]
medical_products
[[6233  312]
 [   0    0]]
search_and_rescue
[[6384  161]
 [   0    0]]
security
[[6439  106]
 [   0    0]]
military
[[6343  202]
 [   0    0]]
child_alone
[[6545]]
water
[[6155  390]
 [   0    0]]
food
[[5865  679]
 [   0    1]]
shelter
[[5980  565]
 [   0    0]]
clothing
[[6449   96]
 [   0    0]]
money
[[6398  147]
 [   0    0]]
missing_people
[[6474   71]
 [   0    0]]
refugees
[[6336  209]
 [   0    0]]
death
[[6249  295]
 [   0    1]]
other_aid
[[5708  837]
 [   0    0]]
infrastructure_related
[[6142  403]
 [   0    0]]
transport
[[6247  298]
 [   0    0]]
buildings
[[6235  310]
 [   0    0]]
electricity
[[6401  144]
 [   0    0]]
tools
[[6505   40]
 [   0    0]]
hospitals
[[6482   63]
 [   0    0]]
shops
[[6510   35]
 [   0    0]]
aid_centers
[[6470   75]
 [   0    0]]
other_infrastructure
[[6281  264]
 [   0    0]]
w

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [129]:
parameters = {
        'tfidf__norm': ['l2'],
        'tfidf__sublinear_tf': [False],
        'tfidf__smooth_idf': [False],
        'clf__estimator__n_estimators': [20, 25, 30, 35, 40],
        'clf__estimator__criterion' :['gini'],
        'clf__estimator__max_depth' : [1,3,5,10,20],
        'clf__estimator__max_features': ['log2']
    }

cv = GridSearchCV(pipeline, parameters)

Now fit the model

In [130]:
cv.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [131]:
y_pred = cv.predict(X_test)
cv.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_depth': 1,
 'clf__estimator__max_features': 'log2',
 'clf__estimator__n_estimators': 20,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': False,
 'tfidf__sublinear_tf': False}

In [132]:
accuracy_score(y_pred, y_test)

0.4430863254392666

In [133]:
for i, label in enumerate(y_test.columns):
    print(label)
    print(confusion_matrix(y_pred[:,i] ,y_test.values[:,i]))

request
[[5455 1090]
 [   0    0]]
offer
[[6516   29]
 [   0    0]]
aid_related
[[3938 2607]
 [   0    0]]
medical_help
[[6063  482]
 [   0    0]]
medical_products
[[6233  312]
 [   0    0]]
search_and_rescue
[[6384  161]
 [   0    0]]
security
[[6439  106]
 [   0    0]]
military
[[6343  202]
 [   0    0]]
child_alone
[[6545]]
water
[[6155  390]
 [   0    0]]
food
[[5865  680]
 [   0    0]]
shelter
[[5980  565]
 [   0    0]]
clothing
[[6449   96]
 [   0    0]]
money
[[6398  147]
 [   0    0]]
missing_people
[[6474   71]
 [   0    0]]
refugees
[[6336  209]
 [   0    0]]
death
[[6249  296]
 [   0    0]]
other_aid
[[5708  837]
 [   0    0]]
infrastructure_related
[[6142  403]
 [   0    0]]
transport
[[6247  298]
 [   0    0]]
buildings
[[6235  310]
 [   0    0]]
electricity
[[6401  144]
 [   0    0]]
tools
[[6505   40]
 [   0    0]]
hospitals
[[6482   63]
 [   0    0]]
shops
[[6510   35]
 [   0    0]]
aid_centers
[[6470   75]
 [   0    0]]
other_infrastructure
[[6281  264]
 [   0    0]]
w

In [96]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

In [97]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', KNeighborsClassifier())
])

In [98]:
parameters = {
        'tfidf__norm': ['l2','l1'],
        'tfidf__sublinear_tf': [True, False],
        'tfidf__smooth_idf': [True, False],
        'clf__leaf_size': [2,3,5,7,10],
        'clf__weights': ['uniform', 'distance']
    }

cv = GridSearchCV(pipeline, parameters)

In [99]:
cv.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [100]:
y_pred = cv.predict(X_test)
cv.best_params_

{'clf__leaf_size': 2,
 'clf__weights': 'distance',
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': False,
 'tfidf__sublinear_tf': True}

In [101]:
accuracy_score(y_pred, y_test)

0.43086325439266615

In [102]:
for i, label in enumerate(y_test.columns):
    print(label)
    print(confusion_matrix(y_pred[:,i] ,y_test.values[:,i]))

request
[[5359  795]
 [  96  295]]
offer
[[6515   29]
 [   1    0]]
aid_related
[[3702 1996]
 [ 236  611]]
medical_help
[[6050  474]
 [  13    8]]
medical_products
[[6222  298]
 [  11   14]]
search_and_rescue
[[6383  157]
 [   1    4]]
security
[[6437  106]
 [   2    0]]
military
[[6339  193]
 [   4    9]]
child_alone
[[6545]]
water
[[6141  339]
 [  14   51]]
food
[[5826  573]
 [  39  107]]
shelter
[[5958  506]
 [  22   59]]
clothing
[[6447   92]
 [   2    4]]
money
[[6397  144]
 [   1    3]]
missing_people
[[6472   71]
 [   2    0]]
refugees
[[6334  207]
 [   2    2]]
death
[[6243  268]
 [   6   28]]
other_aid
[[5664  803]
 [  44   34]]
infrastructure_related
[[6136  397]
 [   6    6]]
transport
[[6238  283]
 [   9   15]]
buildings
[[6228  295]
 [   7   15]]
electricity
[[6399  139]
 [   2    5]]
tools
[[6505   40]
 [   0    0]]
hospitals
[[6482   63]
 [   0    0]]
shops
[[6510   35]
 [   0    0]]
aid_centers
[[6470   75]
 [   0    0]]
other_infrastructure
[[6277  261]
 [   4    3]]
w

### 9. Export your model as a pickle file

In [63]:
disaster_model = 'models/finalized_model.sav'
pickle.dump(cv, open(disaster_model, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.